In [1]:
# Dynohash
# a dynamic POW hash puzzle based on a maximum payload
# this is to maintain low tx fees but still allow complex data to be passed around
# the idea is that lets assume a tx max size of 1kb
# most txs are say 100 bytes
# if an attacker wants to spam the network, if its not 10x as hard to solve a 1kb payload, then they can do 
# much more damage.
# thus...
#
# take the original hashcash idea where H(PAYLOAD + nonce) = 0x000...abc, where the number of leading 0s
# fluctuates. this is the difficulty
# however, in a normal hash, which is usually 64 bytes, thats only 64 degrees of difficulty granularity
# we should smash it down into binary where each byte is 8 bits so that a 64 byte payload now has
# 512 degrees of granularity.
# let's just code it.

import hashlib

In [2]:
h = hashlib.sha256()

In [3]:
import secrets

In [5]:
secrets.token_bytes()

b'm\x97\xd3$Nj\x06\x9b\xa3\xdb\x8a\x0f!\xefD\xf5a1\xc4 f\xd9\x83D\xba\x7f(\x7f24\x04\x9e'

In [10]:
def byte_to_int(b):
    x = 0
    for bb in b:
        x = x * 256 + int(bb)
    return x

In [11]:
bin(byte_to_int(secrets.token_bytes()))

'0b111000101101011111000000010100001011000011011011000001000010101101101001001010110001110110001011110100100101011011111111110010011100000010011010000001001101000001111011100010011001101101100010100101111110100100010011101010000110111101111111010011000'

In [72]:
s = secrets.token_bytes()


while bin(byte_to_int(s))[3:][:5] != '00000':
    s = secrets.token_bytes()
    
print(s)
print(bin(byte_to_int(s)))

b'\x04\n\xe5Oj|h%/A\xd3\xcct\xb3\xe2\xfd\x8d\xa6\xd2\x08\xe4\x13\xe1\xd7/t\x8d\r\xcf[\xb5\xb0'
0b10000001010111001010100111101101010011111000110100000100101001011110100000111010011110011000111010010110011111000101111110110001101101001101101001000001000111001000001001111100001110101110010111101110100100011010000110111001111010110111011010110110000


In [97]:
def mini_dyno(difficulty=10):
    s = secrets.token_bytes()
    while bin(byte_to_int(s))[3:][:difficulty] != ''.join(['0' for x in range(difficulty)]):
        s = secrets.token_bytes()
    return s, bin(byte_to_int(s))[3:]
mini_dyno()

(b'@\x06\xdbA\xeak$\xc1\x9c\xbd\xf9\x1ai\x9a\xa1\xf3\xc3_*ZgB\x98\xc0\xf6\xf5\x16q>\xaaV\xa1',
 '00000000000110110110110100000111101010011010110010010011000001100111001011110111111001000110100110100110011010101000011111001111000011010111110010101001011010011001110100001010011000110000001111011011110101000101100111000100111110101010100101011010100001')

In [132]:
def mini_dyno_payload(payload, difficulty=20):
    # payload + nonce solves the puzzle. payload is a bytestring
    s = secrets.token_bytes()
    h = hashlib.sha256()
    h.update(payload + s)
    while bin(byte_to_int(h.digest()))[3:][:difficulty] != ''.join(['0' for x in range(difficulty)]):
        s = secrets.token_bytes()
        h = hashlib.sha256()
        h.update(payload + s)
    return s, bin(byte_to_int(h.digest()))[3:]

payload = secrets.token_bytes(256)

mini_dyno_payload(payload)

(b'\x1b\xbc\x1aQc\x80I\xb24\xa6\x8f\xf0\x1d\xf0\xeb\xa0X\x96\x9b=\x97\xef\xa4\xc2X\xa43\x007\x1e\xf9\xe0',
 '00000000000000000000111000111101000110001010101111111100011111011100000100110010100110110111100110001011110011101111010011110111110110111011111000101011010001011000010000001010110110010111011101110001101000011101111101100011101111110100010001001111100100')